In [1]:
# End-to-end performance evaluation.  # Run consistency analysis pipeline.

import pandas as pd

from consent.util.default_path import get_data_dir
from ooutil.url_util import get_suffixed_domain

import pandas as pd
pd.options.display.max_rows = 100

from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.consistency.comply_util import get_comply
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans

vio_types = ['omit']

In [2]:
data_dir = get_data_dir('2022-06-03')  # '2022-01-02')
manual_dir = data_dir / 'manual_scan/reject'
assert manual_dir.exists()

In [3]:
# def get_auto_complies():
SCAN_DIRS = [data_dir / 'pref_menu_scan']
auto_cookie_prefs = read_cookie_prefs_in_scans(SCAN_DIRS)
auto_sent_cookies = read_postrej_sent_cookies_in_scans(SCAN_DIRS)
print(f"Number sent cookies read: {len(auto_sent_cookies):,d}")
auto_all_complies = get_comply(auto_cookie_prefs, auto_sent_cookies)
auto_all_complies.head()

Error reading har file /mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-06-03/pref_menu_scan/academyart.edu/postrej_4.har.xz: 'DataFrame' object has no attribute 'request'
Error reading har file /mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-06-03/pref_menu_scan/iguzzini.com/postrej_5.har.xz: 'DataFrame' object has no attribute 'request'
Number sent cookies read: 102,233


100%|██████████| 40/40 [00:20<00:00,  1.91it/s]


,name,value,domain,path,expires,size,httpOnly,secure,session,priority,sameParty,sourceScheme,sourcePort,request_url,site,page_url,sameSite,comply
0,_gcl_au,1.1.1852381578.1654288206,.eppendorf.com,/,1.662064e+09,32,False,False,False,Medium,False,Secure,443,https://www.eppendorf.com/typo3temp/assets/_pr...,eppendorf.com,https://www.eppendorf.com/CH-de/,NaN,omit
1,_ga,GA1.2.719021875.1654288206,.eppendorf.com,/,1.717360e+09,29,False,False,False,Medium,False,Secure,443,https://www.eppendorf.com/typo3temp/assets/_pr...,eppendorf.com,https://www.eppendorf.com/CH-de/,NaN,incorrect
2,OptanonAlertBoxClosed,2022-06-03T20:30:59.158Z,.eppendorf.com,/,1.685824e+09,45,False,False,False,Medium,False,Secure,443,https://www.eppendorf.com/assets/website/image...,eppendorf.com,https://www.eppendorf.com/CH-de/,Lax,comply
3,_ga,GA1.2.719021875.1654288206,.eppendorf.com,/,1.717360e+09,29,False,False,False,Medium,False,Secure,443,https://www.eppendorf.com/CH-de/,eppendorf.com,https://www.eppendorf.com/CH-de/,NaN,incorrect
4,AKA_A2,A,.eppendorf.com,/,1.654292e+09,7,True,True,False,Medium,False,Secure,443,https://www.eppendorf.com/typo3temp/assets/_pr...,eppendorf.com,https://www.eppendorf.com/CH-de/,NaN,comply


In [4]:
# ignore termly for now
auto_complies = auto_all_complies  #[~auto_all_complies.site.isin(['eskill.com'])]
print('number of omitted cookies', auto_complies[auto_complies.comply == 'omit'][['name', 'domain', 'path', 'site']].drop_duplicates().site.value_counts().sum())

number of omitted cookies 399


In [5]:
auto_complies[(auto_complies.site == 'grooveapp.com') & (auto_complies.domain == '.go.groove.co')].page_url.unique()

array(['https://www.groove.co/case-studies/zego-eliminates-sync-errors-with-groove/',
       'https://www.groove.co/case-studies/education-company-increases-student-success-team-productivity-by-45/',
       'https://www.groove.co/case-studies/aquent-improves-sales-productivity-with-groove/',
       'https://www.groove.co/case-studies/sendoso-achieves-sales-marketing-alignment-with-groove/',
       'https://www.groove.co/blog/groove-named-the-highest-rated-enterprise-sales-engagement-platform-in-g2-spring-2022-grid-report/'],
      dtype=object)

In [6]:
# incor = auto_complies[auto_complies.comply == 'incorrect'][['name', 'domain', 'path', 'site', 'comply']].drop_duplicates()
# auto_complies[['name', 'domain', 'path', 'site', 'comply']].drop_duplicates().sort_values('comply')

# Check the detected inconsistencies

In [7]:
from consent.consistency.e2e_eval.resource_reader import get_man_sent_cookies_in_dir
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.e2e_eval.resource_reader'])
from consent.consistency.e2e_eval.resource_reader import get_man_sent_cookies_in_dir

man_sent_cookies, man_cookie_prefs = get_man_sent_cookies_in_dir(manual_dir)
print('Number of sites:', man_sent_cookies.site.nunique())

Number of sites: 40


In [8]:
man_sites = set(man_sent_cookies.site)
auto_sites = set(auto_complies.site) 
if man_sites != auto_sites:
    print(f'WARNING: Manual sites are different from auto sites: {man_sites - auto_sites} {auto_sites - man_sites}')

In [9]:
""" Check 2 conditions: 1. the detected cookie is captured in sent 2. the comply fits with preference."""
from consent.consistency.e2e_eval.resource_reader import get_matches_and_abstains
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.e2e_eval.resource_reader'])
from consent.consistency.e2e_eval.resource_reader import get_matches_and_abstains

sites = set(get_suffixed_domain(afile.stem) for afile in manual_dir.glob('*.har'))
matches, unsents, abstains = get_matches_and_abstains(sites, auto_complies, man_sent_cookies, man_cookie_prefs, vio_types)
print(f'Total cookies: {len(matches) + len(unsents) + len(abstains)}')
unsents.insert(len(unsents.columns)-1, 'site', unsents.pop('site'))
print("Number of manually-unreproduced cookies:", len(unsents))
unsents

Total cookies: 399
Number of manually-unreproduced cookies: 35


,name,domain,path,site
0,fr,.facebook.com,/,eskill.com
1,test_cookie,.doubleclick.net,/,longtailvideo.com
2,test_cookie,.doubleclick.net,/,avetta.com
3,register-prompt,www.psychiatryadvisor.com,/,psychiatryadvisor.com
4,hmmet,www.psychiatryadvisor.com,/,psychiatryadvisor.com
5,dmd-signal-35-218-43E90549-120f5b83-94a5-4e58-...,www.psychiatryadvisor.com,/,psychiatryadvisor.com
6,hmhis,www.psychiatryadvisor.com,/,psychiatryadvisor.com
7,_docereeInit,.doceree.com,/,psychiatryadvisor.com
8,cookies.js_dtest,.remezcla.com,/,remezcla.com
9,visitor_id951682-hash,.pardot.com,/,nic.blue


In [10]:
# auto_complies[auto_complies.site == 'eppendorf.com']

# Analyze abstain cookies (checking omission)

In [11]:
print("Num false positives:", len(matches))
matches

Num false positives: 2


,site,name,domain,path
0,longtailvideo.com,_fbp,.jwplayer.com,/
1,fentybeauty.com,_fbp,.fentybeauty.com,/


In [59]:
# Abstains: detected but not match any
abstain_samples = abstains.sample(30, random_state=3).sort_values(by=['site', 'name', 'domain']).reset_index(drop=True)
abstain_samples 

,name,domain,path,site
0,_ga_28NELPY8BG,.aspentech.com,/,aspentech.com
1,_gat_UA-75008002-8,.avetta.com,/,avetta.com
2,ai_user,embed2.audioburst.com,/,bongino.com
3,uuid2,.adnxs.com,/,bongino.com
4,_gd_visitor,contentsquare.com,/,contentsquare.com
5,_clck,.datarobot.com,/,datarobot.com
6,trd_pws,.datarobot.com,/,datarobot.com
7,CFID,pages.dematic.com,/,dematic.com
8,_dc_gtm_UA-55823836-13,.eppendorf.com,/,eppendorf.com
9,gtm.start.cookie,www.eppendorf.com,/,eppendorf.com


In [28]:
man_cookie_prefs[(man_cookie_prefs['site'] == 'aspentech.com') & (man_cookie_prefs['name'].str.contains('_ga'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'avetta.com') & (man_cookie_prefs['name'].str.contains('_gat_'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'bongino.com') & (man_cookie_prefs['name'].str.contains('ai_'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'bongino.com') & (man_cookie_prefs['name'].str.contains('uuid'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'contentsquare.com') & (man_cookie_prefs['name'].str.contains('_gd'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'datarobot.com') & (man_cookie_prefs['name'].str.contains('clck'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'datarobot.com') & (man_cookie_prefs['name'].str.contains('trd_pws'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'dematic.com') & (man_cookie_prefs['name'].str.contains('CFID'))& (man_cookie_prefs['domain'].str.contains('pages.dematic.com'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'eppendorf.com') & (man_cookie_prefs['name'].str.contains('dc_gtm'))]  # underspecified, number of xxx is too short compared to the actual cookie
man_cookie_prefs[(man_cookie_prefs['site'] == 'eppendorf.com') & (man_cookie_prefs['name'].str.contains('gtm'))]  # under different domain
man_cookie_prefs[(man_cookie_prefs['site'] == 'eppendorf.com') & (man_cookie_prefs['name'].str.contains('popup'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'eskill.com') & (man_cookie_prefs['name'].str.contains('li_gc'))] 

,name,domain,duration,category_id,category,consent_mode,consent,site


In [55]:
man_cookie_prefs[(man_cookie_prefs['site'] == 'essexapartmenthomes.com') & (man_cookie_prefs['name'].str.contains('sxa'))]
man_cookie_prefs[(man_cookie_prefs['site'] == 'fashiongo.net') & (man_cookie_prefs['name'].str.contains('Optanon'))]  # under different domain
man_cookie_prefs[(man_cookie_prefs['site'] == 'fentybeauty.com') & (man_cookie_prefs['name'].str.contains('GlobalE'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'fentybeauty.com') & (man_cookie_prefs['name'].str.contains('attn'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'fentybeauty.com') & (man_cookie_prefs['name'].str.contains('shopify'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'glassdoor.ie') & (man_cookie_prefs['name'].str.contains('Same'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'grooveapp.com') & (man_cookie_prefs['name'].str.contains('Optanon'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'grooveapp.com') & (man_cookie_prefs['name'].str.contains('li_gc'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'lithium.com') & (man_cookie_prefs['name'].str.contains('_gd_s'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'longtailvideo.com') & (man_cookie_prefs['name'].str.contains('visitor'))]  # lack of -hash suffix of the actual cookie
man_cookie_prefs[(man_cookie_prefs['site'] == 'nic.blue') & (man_cookie_prefs['name'].str.contains('hjIn'))]  # lack of -hash suffix of the actual cookie
man_cookie_prefs[(man_cookie_prefs['site'] == 'nic.blue') & (man_cookie_prefs['name'].str.contains('hubspot'))]  
man_cookie_prefs[(man_cookie_prefs['site'] == 'nic.gold') & (man_cookie_prefs['name'].str.contains('fbp'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'nic.gold') & (man_cookie_prefs['name'].str.contains('gcl'))] 
man_cookie_prefs[(man_cookie_prefs['site'] == 'nic.travel') & (man_cookie_prefs['name'].str.contains('hstc'))]  
man_cookie_prefs[(man_cookie_prefs['site'] == 'psychiatryadvisor.com') & (man_cookie_prefs['name'].str.contains('mc'))]  
man_cookie_prefs[(man_cookie_prefs['site'] == 'roca.com') & (man_cookie_prefs['name'].str.contains('f5'))]  
man_cookie_prefs[(man_cookie_prefs['site'] == 'stroke.org.uk') & (man_cookie_prefs['name'].str.contains('hjSession'))]  

,name,domain,duration,category_id,category,consent_mode,consent,site
36,_hjSessionRejected,shop.stroke.org.uk,Session,C0002,Performance Cookies,inactive,False,stroke.org.uk
64,_hjSessionRejected,script.hotjar.com,Session,C0002,Performance Cookies,inactive,False,stroke.org.uk


In [60]:
omit_samples_file = data_dir / 'omit_samples.csv'
abstain_samples.to_csv(omit_samples_file); print(f'Written to {omit_samples_file}')

Written to /mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-06-03/omit_samples.csv


In [14]:
raise

RuntimeError: No active exception to reraise

In [ ]:
incor = auto_complies[auto_complies.comply == 'incorrect'][['name', 'domain', 'path', 'site', 'comply']].drop_duplicates()
incor[incor.name.str.contains('gcl')]

,name,domain,path,site,comply
45,_gcl_au,.eppendorf.com,/,eppendorf.com,incorrect
5949,_gcl_au,.eskill.com,/,eskill.com,incorrect
29106,_gcl_au,.khoros.com,/,lithium.com,incorrect
33885,_gcl_au,.contentsquare.com,/,contentsquare.com,incorrect
38053,_gcl_au,.avetta.com,/,avetta.com,incorrect
40404,_gcl_au,.monsterhunter.com,/,monsterhunter.com,incorrect
44037,_gcl_au,.aspentech.com,/,aspentech.com,incorrect
49976,_gcl_au,.groove.co,/,grooveapp.com,incorrect
56584,_gcl_au,.uwe.ac.uk,/,uwe.ac.uk,incorrect
64559,_gcl_au,.colocrossing.com,/,colocrossing.com,incorrect


In [ ]:
abstains.merge(man_cookie_prefs, on=['name', 'site'], suffixes=['_sent', '_pref'])

,site,name,domain_sent,path,domain_pref,duration,category_id,category,consent_mode,consent


In [ ]:
abstains.merge(auto_cookie_prefs, on=['name', 'site'], suffixes=['_sent', '_pref'])

,site,name,domain_sent,path,domain_pref,duration,category_id,category,consent_mode,lib_name,pattern_name,consent


In [ ]:
# if not man_cookie_prefs[(man_cookie_prefs['name'] == '_GRECAPTCHA')].iloc[0]['consent']:
#     print("Correctly detected")
# else:
#     print("Incorrect detection")

In [ ]:
# Analyze manually-collected incorrect enforcement

In [ ]:
man_sent_cookies[man_sent_cookies.name == 'OptanonConsent'][['name', 'value', 'path', 'domain']].drop_duplicates()

,name,value,path,domain
19,OptanonConsent,isGpcEnabled=0&datestamp=Sat+Jan+08+2022+19%3A...,/,.getsmarter.com
57,OptanonConsent,isGpcEnabled=0&datestamp=Sat+Jan+08+2022+19%3A...,/,.getsmarter.com
33,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+22...,/,.thegrommet.com
36,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+22...,/,.thegrommet.com
22,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+21...,/,.avg.com
...,...,...,...,...
63,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+22...,/,.www.millenniumhotels.com
14,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+19...,/,.asurion.com
30,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+19...,/,.asurion.com
29,OptanonConsent,isIABGlobal=false&datestamp=Sat+Jan+08+2022+19...,/,.payoneer.com


In [ ]:
rej_cookies = man_cookie_prefs[~man_cookie_prefs.consent]
rej_cookies_names = set(rej_cookies['name'])

In [ ]:
# Get sent cookies
# man_sent_cookies[man_sent_cookies['name'].isin(rej_cookies_names)].name.unique()

In [ ]:
from consent.consistency.comply_util import get_comply
man_all_complies = get_comply(man_cookie_prefs, man_sent_cookies)
# man_all_complies # .head()

100%|██████████| 1/1 [00:00<00:00, 97.85it/s]


In [ ]:
auto_cookie_prefs[auto_cookie_prefs.name.str.contains('mf')]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent


In [ ]:
# Manual checking omission
# auto_cookie_prefs[auto_cookie_prefs['name'].str.contains('utmvc')]
auto_cookie_prefs.sort_values('name').reset_index(drop=True)

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,,www.facebook.com,Session,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
1,AMCVS_,budlight.com,Session,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
2,AMCV_,budlight.com,730 days,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
3,ARRAffinity,legends.budlight.com,Session,1,Strictly Necessary Cookies,always active,budlight.com,onetrust,en.json,True
4,CONSENT,youtube.com,6182 days,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
5,IDE,doubleclick.net,390 days,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
6,OptanonAlertBoxClosed,www.budlight.com,365 days,1,Strictly Necessary Cookies,always active,budlight.com,onetrust,en.json,True
7,OptanonConsent,www.budlight.com,365 days,1,Strictly Necessary Cookies,always active,budlight.com,onetrust,en.json,True
8,TDCPM,adsrvr.org,0 days,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False
9,TDID,adsrvr.org,0 days,4,Targeting Cookies and Other Trackers,inactive,budlight.com,onetrust,en.json,False


In [ ]:
man_incorrect = man_all_complies[['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
man_incorrect.head()

,name,domain,path
0,gs_anonymous_id,www.getsmarter.com,/
1,ip_data,www.getsmarter.com,/
2,token,www.getsmarter.com,/
3,guest_token,www.getsmarter.com,/
4,show_banner,www.getsmarter.com,/


In [ ]:
raise

# Analyze incorrect-enforcement cookies

In [ ]:
auto_incor_cookies = auto_all_complies[auto_all_complies['comply'] == 'incorrect'][['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
# auto_incor_cookies

NameError: name 'auto_all_complies' is not defined

In [ ]:
# Add the grouth truth pref & consent to the all_complies
# auto_incor_cookies.merge(cookie_prefs[['name', 'consent', 'category_id', 'domain']], on='name', how='left').sort_values(by='name')
def strip_dot(adomain):
    if adomain.startswith('.'):
        return adomain[1:]
    return adomain
data = []
for row in auto_incor_cookies.to_dict('records'):
    cookie_pref = cookie_prefs[cookie_prefs.domain.str.endswith(strip_dot(row['domain'])) & (cookie_prefs.name == row['name'])]
    datum = row.copy()
    for cookie_pref_row in cookie_pref.to_dict('records'):
        datum.update({'pref_name': cookie_pref_row['name'], 'pref_domain': cookie_pref_row['domain'], 'consent': cookie_pref_row['consent']})
    data.append(datum)
# pd.DataFrame(data)

In [ ]:
# Analyze omit cookies

In [ ]:
auto_omit_cookies = auto_all_complies[auto_all_complies['comply'] == 'omit'][['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
auto_omit_cookies

,name,domain,path
0,shell#lang,www.thule.com,/
1,ASP.NET_SessionId,www.thule.com,/
2,_hjSessionUser_1873881,.thule.com,/
3,_hjSession_1873881,.thule.com,/
4,SC_ANALYTICS_GLOBAL_COOKIE,www.thule.com,/
5,BVImplthule,www.thule.com,/
6,ai_session,www.thule.com,/
7,ai_user,www.thule.com,/
8,_crossdomainGA,.thule.com,/
9,_gcl_au,.thule.com,/


In [ ]:
data = []
for row in auto_omit_cookies.to_dict('records'):
    cookie_pref = cookie_prefs[cookie_prefs.domain.str.endswith(strip_dot(row['domain'])) & (cookie_prefs.name == row['name'])]
    datum = row.copy()
    for cookie_pref_row in cookie_pref.to_dict('records'):
        datum.update({'pref_name': cookie_pref_row['name'], 'pref_domain': cookie_pref_row['domain'], 'consent': cookie_pref_row['consent']})
    data.append(datum)
pd.DataFrame(data)

,name,domain,path
0,shell#lang,www.thule.com,/
1,ASP.NET_SessionId,www.thule.com,/
2,_hjSessionUser_1873881,.thule.com,/
3,_hjSession_1873881,.thule.com,/
4,SC_ANALYTICS_GLOBAL_COOKIE,www.thule.com,/
5,BVImplthule,www.thule.com,/
6,ai_session,www.thule.com,/
7,ai_user,www.thule.com,/
8,_crossdomainGA,.thule.com,/
9,_gcl_au,.thule.com,/


# Analyze comply cookies

In [ ]:
auto_comply_cookies = auto_all_complies[auto_all_complies['comply'] == 'comply'][['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
auto_comply_cookies 

,name,domain,path
0,OptanonAlertBoxClosed,www.thule.com,/
1,OptanonConsent,www.thule.com,/
2,thule#lang,www.thule.com,/


In [ ]:
data = []
for row in auto_comply_cookies.to_dict('records'):
    cookie_pref = cookie_prefs[cookie_prefs.domain.str.endswith(strip_dot(row['domain'])) & (cookie_prefs.name == row['name'])]
    datum = row.copy()
    for cookie_pref_row in cookie_pref.to_dict('records'):
        datum.update({'pref_name': cookie_pref_row['name'], 'pref_domain': cookie_pref_row['domain'], 'consent': cookie_pref_row['consent']})
    data.append(datum)
pd.DataFrame(data)

,name,domain,path,pref_name,pref_domain,consent
0,OptanonAlertBoxClosed,www.thule.com,/,NaN,NaN,NaN
1,OptanonConsent,www.thule.com,/,NaN,NaN,NaN
2,thule#lang,www.thule.com,/,thule#lang,www.thule.com,True


# Analyze ambiguous cookies

In [ ]:
auto_ambiguous_cookies = auto_all_complies[auto_all_complies['comply'] == 'ambiguous'][['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
auto_ambiguous_cookies

,name,domain,path


In [ ]:
auto_incorrect_df = all_complies[['name', 'domain', 'path']].drop_duplicates().reset_index(drop=True)
auto_incorrect = set([tuple((row['name'], row['domain'], row['path'])) for row in auto_incorrect_df.to_dict('records')])

In [ ]:
# Compare the two results.
manual_incorrect = set([tuple((row['name'], row['domain'], row['path'])) for row in manual_incorrect_df.to_dict('records')])

In [ ]:
manual_incorrect.issubset(auto_incorrect)

True

In [ ]:
cookie_prefs

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,__cf_bm,support.thule.com,2914 days,C0001,Strictly Necessary Cookies,always active,thule.com,onetrust,en.json,True
1,__cfduid,support.thule.com,365 days,C0001,Strictly Necessary Cookies,always active,thule.com,onetrust,en.json,True
2,__cfruid,support.thule.com,Session,C0001,Strictly Necessary Cookies,always active,thule.com,onetrust,en.json,True
3,_minicart,www.thule.com,3 days,C0001,Strictly Necessary Cookies,always active,thule.com,onetrust,en.json,True
4,47B023BC-7D14-4624-A93A-45F00E11930B,thule.com,30 days,C0001,Strictly Necessary Cookies,always active,thule.com,onetrust,en.json,True
...,...,...,...,...,...,...,...,...,...,...
14,test_cookie,doubleclick.net,0 days,C0004,Targeting Cookies,active,thule.com,onetrust,en.json,False
15,__atrfs,s7.addthis.com,0 days,C0004,Targeting Cookies,active,thule.com,onetrust,en.json,False
16,__atuvc,s7.addthis.com,396 days,C0004,Targeting Cookies,active,thule.com,onetrust,en.json,False
17,__atuvs,s7.addthis.com,0 days,C0004,Targeting Cookies,active,thule.com,onetrust,en.json,False
